# VLM + Tool Calling 점진적 검증

safari-sample.png 이미지를 사용하여 이미지 인식 → 좌표 추론 → 도구 호출 순서로 능력을 검증한다.

| ID | 테스트 | 측정 능력 | 응답 방식 ~~**|**~~
|----|--------|---------|----------|
| S1 | 원숭이 좌표 인식 | VLM 인식 | 텍스트 |
| S2 | 기린 좌표 인식 | VLM 인식 | 텍스트 |
| S3 | 원숭이 접근 이동 | VLM+추론+tool | move tool |
| S4 | 기린 접근 이동 | VLM+추론+tool | move tool |
| S5 | 원숭이 포획 | VLM+복합tool | move+catch |
| S6 | 종합 미션 | 전체 통합 | move+catch+declare |


In [1]:
# ── Cell 1: 환경 설정 + 경로 ──────────────────────────────────
import os, sys
from dotenv import load_dotenv

load_dotenv()
sys.path.insert(0, os.path.abspath(".."))

print("환경 설정 및 경로 추가 완료")

환경 설정 및 경로 추가 완료


In [ ]:
# ── Cell 2: LLM 연결 ─────────────────────────────────────────
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm_configs = {
    "runpod": {
        "base_url": "https://api.runpod.ai/v2/8iiebdj6zt0fbd/openai/v1",
        "model": "qwen/qwen3-vl-4b-thinking-fp8",
        "api_key": os.getenv("RUNPOD_API_KEY"),
    },
    "local": {
        "base_url": "http://192.168.50.32:8000/v1",
        "model": "Qwen3-VL-2B-Thinking",
        "api_key": "EMPTY",
    },
    "local-emoji": {
        "base_url": "http://192.168.50.32:8000/v1",
        "model": "emoji-lora",
        "api_key": "EMPTY",
    },
    "local-safari": {
        "base_url": "http://192.168.50.32:8000/v1",
        "model": "safari-lora",
        "api_key": "EMPTY",
    },
}

# ── 사용할 설정 선택 ─────────────────────────────────────────
target = "local"
cfg = llm_configs[target]

VLLM_BASE_URL = cfg["base_url"]
MODEL_NAME = cfg["model"]

print(f"현재 활성화된 설정: [{target}]")
print(f"Base URL: {VLLM_BASE_URL}")
print(f"Model Name: {MODEL_NAME}")

llm = ChatOpenAI(
    base_url=VLLM_BASE_URL,
    model=MODEL_NAME,
    temperature=0,
    api_key=cfg["api_key"],
    max_tokens=16000,
)

response = llm.invoke([HumanMessage(content="연결 테스트. 한 문장으로 답해.")])
print(f"\n[연결 확인] {response.content[:200]}")

In [3]:
# ── Cell 3: 도구 정의 + 테스트 하네스 (멀티턴 지원) ─────────────
import json as _json
import time as _time
import re as _re
import base64 as _base64
from typing import Literal

from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage, AIMessage, ToolMessage, SystemMessage,
)
from langchain_core.utils.function_calling import convert_to_openai_tool
from pydantic import BaseModel, Field

# ── 도구 스키마 ────────────────────────────────────────────

class Action(BaseModel):
    direction: Literal["UP", "DOWN", "LEFT", "RIGHT"] = Field(
        description="이동 방향"
    )
    steps: int = Field(description="이동 칸수 (1~3)", ge=1, le=3)

class MoveArgs(BaseModel):
    actions: list[Action] = Field(description="최대 4개 행동을 순서대로 실행")

@tool(args_schema=MoveArgs)
def move(actions: list[Action]) -> str:
    """플레이어를 이동시킨다."""
    return _json.dumps({"moved": True})

class CatchArgs(BaseModel):
    direction: Literal["UP", "DOWN", "LEFT", "RIGHT"] = Field(
        description="포획할 동물이 있는 방향"
    )

@tool(args_schema=CatchArgs)
def catch_animal(direction: str) -> str:
    """인접 타일(상하좌우)의 동물을 포획한다."""
    return _json.dumps({"success": True, "animal": {"emoji": "\ud83d\udc12"}})

class NotepadArgs(BaseModel):
    content: str = Field(
        description="메모장 전체를 덮어쓴다. 유지할 내용도 포함해서 작성해야 한다. 최대 2000자."
    )

@tool(args_schema=NotepadArgs)
def update_notepad(content: str) -> str:
    """메모장 전체를 덮어쓴다."""
    return _json.dumps({"status": "updated"})

class DeclareFoundArgs(BaseModel):
    target: str = Field(description="찾은 타겟 이름")

@tool(args_schema=DeclareFoundArgs)
def declare_found(target: str) -> str:
    """특정 타겟을 찾아서 도달했음을 선언한다."""
    return _json.dumps({"declared": True})

class DeclareDoneArgs(BaseModel):
    reason: str = Field(default="", description="미션 완료 사유")

@tool(args_schema=DeclareDoneArgs)
def declare_done(reason: str = "") -> str:
    """전체 미션이 완료되었음을 선언한다."""
    return _json.dumps({"done": True})

ALL_TOOLS = [move, catch_animal, update_notepad, declare_found, declare_done]

SYSTEM_MSG = (
    "너는 50x50 격자 맵 위의 사파리 에이전트야.\n"
    "이미지는 전체 맵이 아니라, 네 주변 10x10 칸만 보여주는 뷰포트이다.\n"
    "너(P)는 뷰포트의 왼쪽에서 5번째, 위에서 5번째 칸에 있다.\n"
    "반드시 제공된 도구를 사용해서 행동해. 텍스트로 답하지 말고 도구를 호출해.\n"
    "좌표계: x축은 RIGHT(+)/LEFT(-), y축은 DOWN(+)/UP(-)."
)

# ── args list→dict 자동 보정 fallback ────────────────────────

def _to_openai_messages(messages):
    """langchain 메시지 리스트를 OpenAI API 형식으로 변환."""
    result = []
    for m in messages:
        if isinstance(m, SystemMessage):
            result.append({"role": "system", "content": m.content})
        elif isinstance(m, HumanMessage):
            result.append({"role": "user", "content": m.content})
        elif isinstance(m, AIMessage):
            msg = {"role": "assistant", "content": m.content or ""}
            if m.tool_calls:
                msg["tool_calls"] = [
                    {
                        "id": tc["id"],
                        "type": "function",
                        "function": {
                            "name": tc["name"],
                            "arguments": _json.dumps(tc["args"]),
                        },
                    }
                    for tc in m.tool_calls
                ]
            result.append(msg)
        elif isinstance(m, ToolMessage):
            result.append({
                "role": "tool",
                "content": m.content,
                "tool_call_id": m.tool_call_id,
            })
    return result


def _fix_tool_call_args(raw_tool_calls):
    """
    raw OpenAI tool_calls에서 args가 list인 경우 {"actions": list}로 래핑.
    모델이 간헐적으로 wrapper dict를 생략하는 비결정적 포맷 오류를 보정한다.
    """
    fixed = []
    for tc in raw_tool_calls:
        args = _json.loads(tc.function.arguments)
        if isinstance(args, list):
            args = {"actions": args}
            print(f"  [보정] {tc.function.name}: args list → {{\"actions\": [...]}} 래핑")
        fixed.append({
            "name": tc.function.name,
            "args": args,
            "id": tc.id,
            "type": "tool_call",
        })
    return fixed


class _FallbackResponse:
    """pydantic 검증 실패 시 raw OpenAI 응답을 langchain 호환 형태로 감싸는 래퍼."""
    def __init__(self, content, tool_calls, response_metadata):
        self.content = content
        self.tool_calls = tool_calls
        self.response_metadata = response_metadata


def _invoke_with_tool_fallback(llm_instance, tools, messages):
    """
    bind_tools 기반 호출 후, args가 list로 반환되어 pydantic 검증 실패하면
    raw OpenAI client로 재호출하여 args를 dict로 자동 래핑한다.
    """
    llm_with_tools = llm_instance.bind_tools(tools)
    try:
        return llm_with_tools.invoke(messages)
    except Exception as e:
        err_str = str(e)
        if "input_type=list" not in err_str or "dict_type" not in err_str:
            raise

    # ── Fallback: raw OpenAI client 재호출 ──
    print("  [보정] pydantic validation error 감지 → raw client fallback")
    openai_tools = [convert_to_openai_tool(t) for t in tools]
    openai_msgs = _to_openai_messages(messages)

    raw_resp = llm_instance.client.create(
        model=llm_instance.model_name,
        messages=openai_msgs,
        tools=openai_tools,
        temperature=0,
        max_tokens=llm_instance.max_tokens,
    )

    choice = raw_resp.choices[0]
    msg = choice.message

    tool_calls = _fix_tool_call_args(msg.tool_calls) if msg.tool_calls else []
    usage = raw_resp.usage

    return _FallbackResponse(
        content=msg.content or "",
        tool_calls=tool_calls,
        response_metadata={
            "token_usage": {
                "prompt_tokens": usage.prompt_tokens if usage else 0,
                "completion_tokens": usage.completion_tokens if usage else 0,
                "total_tokens": usage.total_tokens if usage else 0,
            }
        },
    )

# ── 공통: 응답 텍스트 출력 헬퍼 ──────────────────────────────

def _print_response_text(resp, passed):
    """응답 텍스트 출력. 실패 시 전체 텍스트, 성공 시 200자 요약."""
    if not resp.content:
        return
    content = resp.content
    if "</think>" in content:
        content = content.split("</think>")[-1].strip()
    if not content:
        return
    if passed:
        print(f"  응답 텍스트: {content[:200]}")
    else:
        print(f"  응답 텍스트 (전체 — tool call 실패 시 모델 출력):")
        for line in content.splitlines():
            print(f"    {line}")

# ── 좌표 추출 헬퍼 ──────────────────────────────────────────

def extract_coordinates(text):
    """텍스트에서 (x, y) 좌표를 추출한다. </think> 태그 이후만 파싱."""
    if "</think>" in text:
        text = text.split("</think>")[-1]
    return [(int(x), int(y)) for x, y in _re.findall(r'\((\d+)\s*,\s*(\d+)\)', text)]

# ── VLM 텍스트 응답 테스트 하네스 ──────────────────────────────

def run_vlm_test(
    test_id: str,
    title: str,
    messages: list,
    validator=None,
    llm_override=None,
) -> dict:
    """도구 없이 텍스트 응답만 받는 VLM 테스트 하네스."""
    llm_instance = llm_override or llm

    print(f"\n{'='*60}")
    print(f"  {test_id}: {title}")
    print(f"{'='*60}")

    start = _time.time()
    try:
        resp = llm_instance.invoke(messages)
        elapsed = _time.time() - start
    except Exception as e:
        elapsed = _time.time() - start
        print(f"\n  FAIL - 호출 실패: {e}")
        return {
            "id": test_id, "title": title, "passed": False,
            "reason": str(e), "tool_calls": [], "tokens": {}, "elapsed": elapsed,
        }

    usage = resp.response_metadata.get("token_usage", {})
    tokens = {
        "prompt": usage.get("prompt_tokens", "?"),
        "completion": usage.get("completion_tokens", "?"),
        "total": usage.get("total_tokens", "?"),
    }

    if validator:
        passed, reason = validator(resp)
    else:
        passed = bool(resp.content and resp.content.strip())
        reason = "텍스트 응답 존재" if passed else "응답 없음"

    status = "PASS" if passed else "FAIL"

    print(f"\n  상태: {status}")
    print(f"  사유: {reason}")
    print(f"  소요시간: {elapsed:.1f}s")
    print(f"  토큰: prompt={tokens['prompt']}, completion={tokens['completion']}, total={tokens['total']}")

    _print_response_text(resp, passed)

    return {
        "id": test_id, "title": title, "passed": passed,
        "reason": reason, "tool_calls": [],
        "tokens": tokens, "elapsed": elapsed,
    }

# ── 싱글턴 테스트 하네스 ──────────────────────────────────────

def run_test(
    test_id: str,
    title: str,
    messages: list,
    tools: list = None,
    validator=None,
    llm_override=None,
) -> dict:
    """단일 테스트 실행 및 결과 출력. llm_override로 별도 LLM 인스턴스 사용 가능."""
    if tools is None:
        tools = ALL_TOOLS
    llm_instance = llm_override or llm

    print(f"\n{'='*60}")
    print(f"  {test_id}: {title}")
    print(f"{'='*60}")

    start = _time.time()
    try:
        resp = _invoke_with_tool_fallback(llm_instance, tools, messages)
        elapsed = _time.time() - start
    except Exception as e:
        elapsed = _time.time() - start
        print(f"\n  FAIL - 호출 실패: {e}")
        return {
            "id": test_id, "title": title, "passed": False,
            "reason": str(e), "tool_calls": [], "tokens": {}, "elapsed": elapsed,
        }

    usage = resp.response_metadata.get("token_usage", {})
    tokens = {
        "prompt": usage.get("prompt_tokens", "?"),
        "completion": usage.get("completion_tokens", "?"),
        "total": usage.get("total_tokens", "?"),
    }

    tool_calls = resp.tool_calls or []

    if validator:
        passed, reason = validator(resp, tool_calls)
    else:
        passed = len(tool_calls) > 0
        reason = "tool call 존재" if passed else "tool call 없음"

    status = "PASS" if passed else "FAIL"

    print(f"\n  상태: {status}")
    print(f"  사유: {reason}")
    print(f"  소요시간: {elapsed:.1f}s")
    print(f"  토큰: prompt={tokens['prompt']}, completion={tokens['completion']}, total={tokens['total']}")
    print(f"  Tool calls ({len(tool_calls)}개):")
    for i, tc in enumerate(tool_calls):
        print(f"    [{i}] {tc['name']}({_json.dumps(tc['args'], ensure_ascii=False)})")

    _print_response_text(resp, passed)

    return {
        "id": test_id, "title": title, "passed": passed,
        "reason": reason, "tool_calls": tool_calls,
        "tokens": tokens, "elapsed": elapsed,
    }

# ── 멀티턴 테스트 하네스 ──────────────────────────────────────

def run_multiturn_test(
    test_id: str,
    title: str,
    turns: list[dict],
    tools: list = None,
    validator=None,
    llm_override=None,
) -> dict:
    """
    멀티턴 테스트 실행.
    turns: list of {"user_msg": str, "tool_results": dict[tool_name, str]}
      - user_msg: 해당 턴의 사용자 메시지 (첫 턴은 시스템 메시지 자동 포함)
      - tool_results: 이전 턴의 tool call에 대한 모의 결과 (첫 턴은 무시)
    validator: (all_turn_tool_calls: list[list[dict]], messages) -> (bool, str)
    """
    if tools is None:
        tools = ALL_TOOLS
    llm_instance = llm_override or llm

    print(f"\n{'='*60}")
    print(f"  {test_id}: {title}")
    print(f"{'='*60}")

    messages = [SystemMessage(content=SYSTEM_MSG)]

    all_turn_tool_calls = []
    all_turn_responses = []
    total_tokens = {"prompt": 0, "completion": 0, "total": 0}
    start = _time.time()

    for turn_idx, turn in enumerate(turns):
        print(f"\n  -- Turn {turn_idx + 1} --")

        # 이전 턴의 tool result 추가 (turn_idx > 0)
        if turn_idx > 0 and all_turn_tool_calls[turn_idx - 1]:
            prev_tc = all_turn_tool_calls[turn_idx - 1]
            tool_result_map = turn.get("tool_results", {})
            for tc in prev_tc:
                result_content = tool_result_map.get(
                    tc["name"],
                    _json.dumps({"ok": True}),
                )
                messages.append(ToolMessage(
                    content=result_content,
                    tool_call_id=tc["id"],
                ))

        messages.append(HumanMessage(content=turn["user_msg"]))

        try:
            resp = _invoke_with_tool_fallback(llm_instance, tools, messages)
        except Exception as e:
            print(f"    FAIL - 호출 실패: {e}")
            all_turn_tool_calls.append([])
            all_turn_responses.append(None)
            messages.append(AIMessage(content=str(e)))
            continue

        usage = resp.response_metadata.get("token_usage", {})
        for k in total_tokens:
            key = f"{k}_tokens"
            total_tokens[k] += usage.get(key, 0)

        tc = resp.tool_calls or []
        all_turn_tool_calls.append(tc)
        all_turn_responses.append(resp)

        # _FallbackResponse는 AIMessage가 아니므로 messages에 AIMessage로 변환하여 추가
        if isinstance(resp, _FallbackResponse):
            ai_msg = AIMessage(content=resp.content)
            # tool_calls는 pydantic 검증 우회 위해 additional_kwargs에만 저장
            ai_msg.additional_kwargs["tool_calls"] = [
                {
                    "id": t["id"],
                    "type": "function",
                    "function": {
                        "name": t["name"],
                        "arguments": _json.dumps(t["args"]),
                    },
                }
                for t in tc
            ]
            messages.append(ai_msg)
        else:
            messages.append(resp)

        for i, t in enumerate(tc):
            print(f"    [{i}] {t['name']}({_json.dumps(t['args'], ensure_ascii=False)})")
        if not tc:
            print(f"    (tool call 없음)")
            # 실패한 턴의 응답 텍스트 전체 출력
            _print_response_text(resp, False)

    elapsed = _time.time() - start

    if validator:
        passed, reason = validator(all_turn_tool_calls, messages)
    else:
        passed = all(len(tc) > 0 for tc in all_turn_tool_calls)
        reason = "모든 턴 tool call 존재" if passed else "일부 턴 tool call 없음"

    status = "PASS" if passed else "FAIL"
    print(f"\n  상태: {status}")
    print(f"  사유: {reason}")
    print(f"  소요시간: {elapsed:.1f}s")
    print(f"  누적 토큰: {total_tokens}")

    flat_calls = [tc for turn_tc in all_turn_tool_calls for tc in turn_tc]
    return {
        "id": test_id, "title": title, "passed": passed,
        "reason": reason, "tool_calls": flat_calls,
        "turn_tool_calls": all_turn_tool_calls,
        "tokens": total_tokens, "elapsed": elapsed,
    }

results = []

print("테스트 하네스 및 도구 정의 완료")
print(f"사용 가능 도구: {[t.name for t in ALL_TOOLS]}")

테스트 하네스 및 도구 정의 완료
사용 가능 도구: ['move', 'catch_animal', 'update_notepad', 'declare_found', 'declare_done']


## 테스트 아키텍처

**이미지 기반 점진적 검증:** safari-sample.png (10x10 뷰포트) 하나로 모든 테스트 수행.

**단계:**
1. **인식** (S1-S2): 이미지에서 동물 좌표를 텍스트로 답변 — VLM perception 검증
2. **추론+도구** (S3-S4): 좌표 차이 계산 후 move 도구 호출 — 추론+tool 검증
3. **복합 도구** (S5): 이동 + 포획 — multi-tool 검증
4. **통합** (S6): 이동 + 포획 + 선언 + 메모 — full workflow 검증

**좌표계:** `UP=y-1, DOWN=y+1, RIGHT=x+1, LEFT=x-1` (실제 게임과 통일)

**Ground Truth:** Player(25,25), Monkey(29,23), Giraffe(29,28) — 허용 오차 ±2


In [4]:
# ── Cell 5: 이미지 로드 + Ground Truth 정의 ─────────────────────

# 이미지 로드
sample_img_path = "images/safari-sample.png"
with open(sample_img_path, "rb") as f:
    _b64 = _base64.b64encode(f.read()).decode()
IMAGE_URL = f"data:image/png;base64,{_b64}"

# Ground Truth
PLAYER_POS = (25, 25)
MONKEY_POS = (29, 23)   # 빨간배경, 뷰포트(8,2)
GIRAFFE_POS = (29, 28)  # 노란배경, 뷰포트(8,7)
COORD_TOLERANCE = 2     # 좌표 허용 오차

# VLM 인식 테스트용 시스템 프롬프트 (도구 호출 지시 없음)
VLM_SYSTEM_MSG = (
    "너는 50x50 격자 맵의 시야 분석가야.\n"
    "이 이미지는 전체 맵이 아니라, 플레이어 주변 10x10 칸만 보여주는 뷰포트이다.\n"
    "뷰포트는 10열(가로) × 10행(세로)의 격자이다.\n"
    "플레이어(P)는 뷰포트에서 왼쪽에서 5번째, 위에서 5번째 칸에 있다.\n"
    "좌표 변환법: 뷰포트에서 P 기준으로 오브젝트가 오른쪽으로 n칸이면 x좌표는 플레이어x + n, 아래로 m칸이면 y좌표는 플레이어y + m이다.\n"
    "좌표계: x축은 RIGHT(+)/LEFT(-), y축은 DOWN(+)/UP(-)."
)

# 도구 호출 테스트용 게임 컨텍스트 템플릿
def make_game_context(mission, step=1, max_steps=10):
    return (
        f"Mission: {mission}\n"
        f"Step: {step}/{max_steps}\n"
        f"Position: ({PLAYER_POS[0]}, {PLAYER_POS[1]})\n"
        "Found Targets: (none)\n"
        "Notepad: (empty)\n"
    )

print(f"이미지 로드 완료: {sample_img_path} ({len(_b64)} bytes base64)")
print(f"Ground Truth: Player={PLAYER_POS}, Monkey={MONKEY_POS}, Giraffe={GIRAFFE_POS}")
print(f"허용 오차: ±{COORD_TOLERANCE}")

이미지 로드 완료: images/safari-sample.png (38864 bytes base64)
Ground Truth: Player=(25, 25), Monkey=(29, 23), Giraffe=(29, 28)
허용 오차: ±2


## 기초 시각 인식 진단 (Pre-S1)

좌표 추출·tool calling 이전에, 모델이 이미지를 **볼 수 있는지** 자체를 먼저 검증한다.
도구 없이 텍스트 응답만 받으며, `max_tokens=2000`으로 제한하여 thinking 루프를 방지한다.

| ID | 질문 | 통과 기준 |
|----|------|----------|
| Q1 | 아이콘/이모지 나열 | 원숭이·기린·나무·P 중 하나 이상 언급 |
| Q2 | P 기준 빨간 배경 방향 | "우상" 또는 "오른쪽 위" 계열 |
| Q3 | P 기준 노란 배경 방향 | "우하" 또는 "오른쪽 아래" 계열 |

In [5]:
# ── 기초 시각 인식 진단 (Q1~Q3) ─────────────────────────────────
# 도구 없이 텍스트 응답만 받아 모델의 시각 인식 능력 자체를 검증한다.
# max_tokens=2000 으로 thinking 루프 방지.

import re as _re

diag_llm = ChatOpenAI(
    base_url=VLLM_BASE_URL,
    model=MODEL_NAME,
    temperature=0,
    api_key=cfg["api_key"],
    max_tokens=2000,
)

diag_results = []

# ── Q1: 아이콘 나열 ──────────────────────────────────────────

def validate_q1(resp):
    text = resp.content
    if "</think>" in text:
        text = text.split("</think>")[-1]
    text_lower = text.lower()
    keywords = ["원숭이", "monkey", "🐒", "🐵",
                 "기린", "giraffe", "🦒",
                 "나무", "tree", "🌲", "🌳", "🌴",
                 "p", "플레이어", "player"]
    found = [k for k in keywords if k in text_lower]
    if found:
        return True, f"시각 인식 작동 — 언급된 키워드: {found}"
    return False, "아이콘/이모지 관련 키워드 없음"

r = run_vlm_test(
    test_id="Q1",
    title="아이콘 나열 (시각 인식 기초)",
    messages=[
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": "이 이미지에 어떤 아이콘/이모지가 보이는지 나열해."},
        ]),
    ],
    validator=validate_q1,
    llm_override=diag_llm,
)
diag_results.append(r)

# ── Q2: 빨간 배경 방향 ───────────────────────────────────────

def validate_q2(resp):
    text = resp.content
    if "</think>" in text:
        text = text.split("</think>")[-1]
    # "우상", "오른쪽 위", "right-up", "upper right" 등
    patterns = [
        r"우상", r"오른쪽\s*위", r"우측\s*상", r"상\s*우",
        r"위\s*오른", r"upper\s*right", r"right\s*up",
        r"top\s*right", r"right.*top", r"up.*right",
    ]
    for p in patterns:
        if _re.search(p, text, _re.IGNORECASE):
            return True, f"정확 — '{p}' 패턴 매치"
    return False, f"'우상/오른쪽 위' 계열 답변 없음"

r = run_vlm_test(
    test_id="Q2",
    title="빨간 배경 방향 (P 기준)",
    messages=[
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": (
                "이미지에서 P 글자가 보일 거야. "
                "P 기준으로 빨간색 배경은 어느 방향에 있어? "
                "(상/하/좌/우/우상/우하/좌상/좌하 중 하나로 답해)"
            )},
        ]),
    ],
    validator=validate_q2,
    llm_override=diag_llm,
)
diag_results.append(r)

# ── Q3: 노란 배경 방향 ───────────────────────────────────────

def validate_q3(resp):
    text = resp.content
    if "</think>" in text:
        text = text.split("</think>")[-1]
    # "우하", "오른쪽 아래", "right-down", "lower right" 등
    patterns = [
        r"우하", r"오른쪽\s*아래", r"우측\s*하", r"하\s*우",
        r"아래\s*오른", r"lower\s*right", r"right\s*down",
        r"bottom\s*right", r"right.*bottom", r"down.*right",
    ]
    for p in patterns:
        if _re.search(p, text, _re.IGNORECASE):
            return True, f"정확 — '{p}' 패턴 매치"
    return False, f"'우하/오른쪽 아래' 계열 답변 없음"

r = run_vlm_test(
    test_id="Q3",
    title="노란 배경 방향 (P 기준)",
    messages=[
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": (
                "이미지에서 P 글자가 보일 거야. "
                "P 기준으로 노란색 배경은 어느 방향에 있어? "
                "(상/하/좌/우/우상/우하/좌상/좌하 중 하나로 답해)"
            )},
        ]),
    ],
    validator=validate_q3,
    llm_override=diag_llm,
)
diag_results.append(r)

# ── 진단 결과 요약 ────────────────────────────────────────────

print(f"\n{'='*60}")
print(f"  기초 시각 인식 진단 결과")
print(f"{'='*60}")

for r in diag_results:
    status = "PASS ✓" if r["passed"] else "FAIL ✗"
    print(f"  {r['id']}: {r['title']} → {status}")
    print(f"       {r['reason']}")

passed = sum(1 for r in diag_results if r["passed"])
print(f"\n  진단 통과: {passed}/{len(diag_results)}")

if passed == 0:
    print("\n  ⚠ 모델이 이미지를 전혀 인식하지 못합니다. 좌표/도구 테스트 진행 불필요.")
elif passed < len(diag_results):
    print("\n  ⚠ 부분적 인식. S1 이후 테스트 결과 해석 시 주의 필요.")


  Q1: 아이콘 나열 (시각 인식 기초)

  상태: PASS
  사유: 시각 인식 작동 — 언급된 키워드: ['나무', 'p']
  소요시간: 6.8s
  토큰: prompt=257, completion=808, total=1065
  응답 텍스트: 이 이미지에 보이는 아이콘/이모지로 다음과 같이 나열할 수 있습니다:  

1. **녹색 나무 아이콘** (multiple)  
2. **猩猩 아이콘** (빨간색 사각형 안에 위치한)  
3. **P 아이콘** (파란색 원 안에 위치한)  
4. **소리개 아이콘** (노란색 사각형 안에 위치한)

  Q2: 빨간 배경 방향 (P 기준)

  상태: PASS
  사유: 정확 — 'upper\s*right' 패턴 매치
  소요시간: 17.1s
  토큰: prompt=294, completion=2000, total=2294
  응답 텍스트: Got it, let's try to figure out where the red square is relative to the P. First, I need to visualize the grid. Let's list out the positions. The P is in the center, right? Let's see the grid. The red

  Q3: 노란 배경 방향 (P 기준)

  상태: PASS
  사유: 정확 — 'bottom\s*right' 패턴 매치
  소요시간: 17.3s
  토큰: prompt=292, completion=2000, total=2292
  응답 텍스트: Got it, let's try to figure out where the yellow background is relative to the P. First, I need to locate the P. In the image, there's a blue circle with a P in the middle. Now, the yellow background 

  기초 시각 인식

In [ ]:
# ── S1: 원숭이 좌표 인식 (VLM perception, 텍스트) ────────────────
# 측정 능력: VLM 인식
# 이미지에서 빨간배경 동물의 절대 좌표를 텍스트로 답변

def validate_s1(resp):
    coords = extract_coordinates(resp.content)
    if not coords:
        return False, "좌표 추출 실패"
    # 추출된 좌표 중 MONKEY_POS에 가까운 것이 있는지 확인
    for x, y in coords:
        if abs(x - MONKEY_POS[0]) <= COORD_TOLERANCE and abs(y - MONKEY_POS[1]) <= COORD_TOLERANCE:
            return True, f"좌표 ({x},{y}) ≈ 정답 {MONKEY_POS} (오차 ±{COORD_TOLERANCE} 이내)"
    return False, f"추출 좌표 {coords} ≠ 정답 {MONKEY_POS} (오차 ±{COORD_TOLERANCE} 초과)"

result = run_vlm_test(
    test_id="S1",
    title="원숭이 좌표 인식 (VLM perception)",
    messages=[
        SystemMessage(content=VLM_SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": (
                "플레이어(P)의 절대 좌표는 (25,25)이다.\n"
                "이 이미지는 P 주변 10x10 뷰포트이다. P는 뷰포트의 왼쪽에서 5번째, 위에서 5번째 칸에 있다.\n"
                "예시: P 바로 오른쪽 칸의 절대좌표는 (26,25), P 바로 아래 칸은 (25,26)이다.\n"
                "빨간색 배경 위의 동물이 P 기준으로 뷰포트에서 몇 칸 떨어져 있는지 세고, 절대 좌표를 (x, y) 형태로 답해."
            )},
        ]),
    ],
    validator=validate_s1,
)
results.append(result)

In [ ]:
# ── S2: 기린 좌표 인식 (VLM perception, 텍스트) ─────────────────
# 측정 능력: VLM 인식
# 이미지에서 노란배경 동물의 절대 좌표를 텍스트로 답변

def validate_s2(resp):
    coords = extract_coordinates(resp.content)
    if not coords:
        return False, "좌표 추출 실패"
    for x, y in coords:
        if abs(x - GIRAFFE_POS[0]) <= COORD_TOLERANCE and abs(y - GIRAFFE_POS[1]) <= COORD_TOLERANCE:
            return True, f"좌표 ({x},{y}) ≈ 정답 {GIRAFFE_POS} (오차 ±{COORD_TOLERANCE} 이내)"
    return False, f"추출 좌표 {coords} ≠ 정답 {GIRAFFE_POS} (오차 ±{COORD_TOLERANCE} 초과)"

result = run_vlm_test(
    test_id="S2",
    title="기린 좌표 인식 (VLM perception)",
    messages=[
        SystemMessage(content=VLM_SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": (
                "플레이어(P)의 절대 좌표는 (25,25)이다.\n"
                "이 이미지는 P 주변 10x10 뷰포트이다. P는 뷰포트의 왼쪽에서 5번째, 위에서 5번째 칸에 있다.\n"
                "예시: P 바로 오른쪽 칸의 절대좌표는 (26,25), P 바로 아래 칸은 (25,26)이다.\n"
                "노란색 배경 위의 동물이 P 기준으로 뷰포트에서 몇 칸 떨어져 있는지 세고, 절대 좌표를 (x, y) 형태로 답해."
            )},
        ]),
    ],
    validator=validate_s2,
)
results.append(result)

In [ ]:
# ── S3: 원숭이 접근 move (VLM + single tool) ────────────────────
# 측정 능력: VLM + 추론 + tool
# 이미지를 보고 빨간배경 동물에게 접근하는 move 호출
# 검증: move 호출됨, net dx > 0 (RIGHT), net dy < 0 (UP)

game_ctx_s3 = make_game_context("빨간색 배경의 동물에게 접근하라.")

def validate_s3(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, f"move 호출 없음 (호출된 도구: {[tc['name'] for tc in tool_calls]})"

    dx, dy = 0, 0
    for mc in move_calls:
        for a in mc["args"].get("actions", []):
            d, s = a["direction"], a["steps"]
            if d == "RIGHT": dx += s
            elif d == "LEFT": dx -= s
            elif d == "DOWN": dy += s
            elif d == "UP": dy -= s

    issues = []
    if dx <= 0:
        issues.append(f"dx={dx:+d} (expected >0, RIGHT)")
    if dy >= 0:
        issues.append(f"dy={dy:+d} (expected <0, UP)")

    if not issues:
        return True, f"move 방향 정확: dx={dx:+d}(RIGHT), dy={dy:+d}(UP)"
    return False, ", ".join(issues)

result = run_test(
    test_id="S3",
    title="원숭이 접근 move (VLM+추론+tool)",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": game_ctx_s3 + "\n이미지에서 P 위치를 기준으로 빨간색 배경 동물의 방향과 거리를 파악해서 move 도구를 호출해."},
        ]),
    ],
    tools=[move],
    validator=validate_s3,
)
results.append(result)

In [ ]:
# ── S4: 기린 접근 move (VLM + single tool) ──────────────────────
# 측정 능력: VLM + 추론 + tool
# 이미지를 보고 노란배경 동물에게 접근하는 move 호출
# 검증: move 호출됨, net dx > 0 (RIGHT), net dy > 0 (DOWN)

game_ctx_s4 = make_game_context("노란색 배경의 동물에게 접근하라.")

def validate_s4(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, f"move 호출 없음 (호출된 도구: {[tc['name'] for tc in tool_calls]})"

    dx, dy = 0, 0
    for mc in move_calls:
        for a in mc["args"].get("actions", []):
            d, s = a["direction"], a["steps"]
            if d == "RIGHT": dx += s
            elif d == "LEFT": dx -= s
            elif d == "DOWN": dy += s
            elif d == "UP": dy -= s

    issues = []
    if dx <= 0:
        issues.append(f"dx={dx:+d} (expected >0, RIGHT)")
    if dy <= 0:
        issues.append(f"dy={dy:+d} (expected >0, DOWN)")

    if not issues:
        return True, f"move 방향 정확: dx={dx:+d}(RIGHT), dy={dy:+d}(DOWN)"
    return False, ", ".join(issues)

result = run_test(
    test_id="S4",
    title="기린 접근 move (VLM+추론+tool)",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": game_ctx_s4 + "\n이미지에서 P 위치를 기준으로 노란색 배경 동물의 방향과 거리를 파악해서 move 도구를 호출해."},
        ]),
    ],
    tools=[move],
    validator=validate_s4,
)
results.append(result)

In [ ]:
# ── S5: 원숭이 포획 (VLM + multi-tool) ──────────────────────────
# 측정 능력: VLM + 복합 tool
# 이미지를 보고 빨간배경 동물에게 접근 + 포획
# 검증: move + catch_animal 모두 호출됨

game_ctx_s5 = make_game_context("빨간색 배경의 동물을 포획하라.")

def validate_s5(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    names = [tc["name"] for tc in tool_calls]
    has_move = "move" in names
    has_catch = "catch_animal" in names

    issues = []
    if not has_move:
        issues.append("move 누락")
    if not has_catch:
        issues.append("catch_animal 누락")

    if not issues:
        return True, f"move + catch_animal 모두 호출 ({len(tool_calls)}개 tool calls)"
    return False, f"{', '.join(issues)} (호출된 도구: {names})"

result = run_test(
    test_id="S5",
    title="원숭이 포획 (VLM+복합tool)",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": game_ctx_s5 + "\n이미지에서 P 위치를 기준으로 빨간색 배경 동물의 방향과 거리를 파악해서 접근하고 포획해."},
        ]),
    ],
    tools=[move, catch_animal],
    validator=validate_s5,
)
results.append(result)

In [ ]:
# ── S6: 종합 미션 (full workflow) ────────────────────────────────
# 측정 능력: 전체 통합
# 이미지를 보고 원숭이 포획 + 발견 선언 + 메모장 기록
# 검증: move, catch_animal, declare_found, update_notepad 중 3개 이상 호출

game_ctx_s6 = make_game_context("빨간색 배경의 원숭이를 포획하고 발견을 선언하라.")

def validate_s6(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    names = set(tc["name"] for tc in tool_calls)
    required = {"move", "catch_animal", "declare_found", "update_notepad"}
    matched = names & required
    missing = required - names

    if len(matched) >= 3:
        return True, f"{len(matched)}/4 도구 호출: {matched} (누락: {missing or 'none'})"
    return False, f"{len(matched)}/4 도구만 호출: {matched}, 누락: {missing}"

result = run_test(
    test_id="S6",
    title="종합 미션 (full workflow)",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": IMAGE_URL}},
            {"type": "text", "text": (
                game_ctx_s6 +
                "\n이미지에서 P 위치를 기준으로 빨간색 배경 원숭이의 방향과 거리를 파악해서 "
                "포획하고 발견을 선언해. 메모장에 상황을 기록해."
            )},
        ]),
    ],
    tools=ALL_TOOLS,
    validator=validate_s6,
)
results.append(result)

In [ ]:
# ── 결과 요약표 ─────────────────────────────────────────────

print(f"\n{'='*70}")
print(f"  VLM + Tool Calling 점진적 검증 결과")
print(f"{'='*70}")

ability_map = {
    "S1": "VLM 인식",
    "S2": "VLM 인식",
    "S3": "VLM+추론+tool",
    "S4": "VLM+추론+tool",
    "S5": "VLM+복합tool",
    "S6": "전체 통합",
}

stage_map = {
    "S1": "인식",
    "S2": "인식",
    "S3": "추론+도구",
    "S4": "추론+도구",
    "S5": "복합도구",
    "S6": "통합",
}

header = f"  {'ID':<6} {'테스트':<28} {'능력 단계':<12} {'측정 능력':<16} {'결과':<6} {'토큰':>8} {'시간':>7}"
print(f"\n{header}")
print(f"  {'-'*6} {'-'*28} {'-'*12} {'-'*16} {'-'*6} {'-'*8} {'-'*7}")

for r in results:
    tid = r["id"]
    title = r["title"][:26]
    stage = stage_map.get(tid, "")
    ability = ability_map.get(tid, "")
    status = "PASS" if r["passed"] else "FAIL"
    tok = str(r["tokens"].get("total", "?"))
    elapsed = f"{r['elapsed']:.1f}s"
    print(f"  {tid:<6} {title:<28} {stage:<12} {ability:<16} {status:<6} {tok:>8} {elapsed:>7}")

passed = sum(1 for r in results if r["passed"])
total = len(results)
print(f"\n  통과: {passed}/{total}")

# 단계별 통과율
print(f"\n  단계별 통과율:")
for stage_name in ["인식", "추론+도구", "복합도구", "통합"]:
    stage_results = [r for r in results if stage_map.get(r["id"]) == stage_name]
    if stage_results:
        stage_passed = sum(1 for r in stage_results if r["passed"])
        print(f"    {stage_name}: {stage_passed}/{len(stage_results)}")